# Task 3 — End-to-End Data Science Project
## House Price Prediction API (Flask)

This notebook implements a full data science workflow:
1. Load dataset (California Housing)
2. Preprocess features and train a model (RandomForest)
3. Save model and preprocessing pipeline
4. Provide a Flask app (`flask_app.py`) that loads the saved model and serves a `/predict` endpoint

The notebook is self-contained and uses sklearn's built-in dataset so no external downloads are required.

In [ ]:
# 1. Imports
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import joblib

print('Libraries imported')

In [ ]:
# 2. Load data
cal = fetch_california_housing(as_frame=True)
df = cal.frame.copy()
print('Data shape:', df.shape)
df.head()

In [ ]:
# Show a small preview using the environment display helper
import caas_jupyter_tools as cjt
cjt.display_dataframe_to_user('California Housing - sample', df.head(10))

In [ ]:
# 3. Prepare features and target
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

# For simplicity we will treat all features as numeric
numeric_features = X.columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features)
])

# Full pipeline with model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=50, random_state=42))
])

# Split
a_train, a_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape:', a_train.shape, 'Test shape:', a_test.shape)

In [ ]:
# 4. Train the model
model.fit(a_train, y_train)

# Predictions and evaluation
preds = model.predict(a_test)
rmse = mean_squared_error(y_test, preds, squared=False)
r2 = r2_score(y_test, preds)
print(f'RMSE: {rmse:.4f}, R2: {r2:.4f}')

In [ ]:
# 5. Save the model and preprocessing pipeline
model_save_path = str(r'/mnt/data/house_price_model.joblib')
joblib.dump(model, model_save_path)
print('Saved model to', model_save_path)

# Also save a small sample of feature names for the API
features_meta = {'feature_names': numeric_features}
with open('/mnt/data/features_meta.json', 'w') as f:
    json.dump(features_meta, f)
print('Saved features_meta.json')

In [ ]:
# 6. Example: Using the saved model to predict on a single sample
loaded = joblib.load('/mnt/data/house_price_model.joblib')
sample = a_test.iloc[[0]]
print('Raw sample:\n', sample.to_dict(orient='records'))
print('\nModel prediction:', loaded.predict(sample)[0])

## Flask app

A Flask app has been created at `/mnt/data/flask_app.py`. It provides a `/predict` endpoint that accepts JSON with the feature values and returns a predicted median house value.

Example curl request:

```
curl -X POST http://127.0.0.1:5000/predict -H "Content-Type: application/json" -d '{"data": [{"MedInc": 8.3252, "HouseAge": 41.0, "AveRooms": 6.98, "AveBedrms": 1.023, "Population": 322.0, "AveOccup": 2.5556, "Latitude": 37.88, "Longitude": -122.23}] }'
```

The endpoint expects a JSON object with key `data` which is a list of records (dictionaries) matching the feature names.

In [ ]:
# 7. Show contents of the Flask app file for reference
flask_code = '''
from flask import Flask, request, jsonify
import joblib
import json
import pandas as pd

app = Flask(__name__)

# Load model and feature metadata
model = joblib.load('/mnt/data/house_price_model.joblib')
with open('/mnt/data/features_meta.json', 'r') as f:
    meta = json.load(f)
FEATURE_NAMES = meta['feature_names']

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    if not data or 'data' not in data:
        return jsonify({'error': "JSON with 'data' key required. Example: {'data': [{...}] }"}), 400
    try:
        df = pd.DataFrame(data['data'])
        # Ensure columns order
        df = df[FEATURE_NAMES]
        preds = model.predict(df)
        return jsonify({'predictions': preds.tolist()})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)
'''

# Write the flask app to disk
with open('/mnt/data/flask_app.py', 'w') as f:
    f.write(flask_code)

print('Wrote /mnt/data/flask_app.py')

# Also write the flask_code variable into the notebook output for user readability
flask_code[:1000]

## How to run the Flask app locally

1. Ensure the required packages are installed: `pip install flask scikit-learn joblib pandas`
2. Run the Flask app: `python /mnt/data/flask_app.py`
3. Send a POST request to `http://127.0.0.1:5000/predict` with a JSON payload as shown above.

The trained model file is saved at `/mnt/data/house_price_model.joblib` and feature metadata at `/mnt/data/features_meta.json`.

In [ ]:
# 8. Provide the paths of generated files for convenience
{
  'notebook': str('/mnt/data/task3_end_to_end_deployment.ipynb'),
  'flask_app': str('/mnt/data/flask_app.py'),
  'model': str('/mnt/data/house_price_model.joblib'),
  'features_meta': '/mnt/data/features_meta.json'
}

If you'd like, I can:
- Create a `requirements.txt` file for easy setup
- Create a Dockerfile to containerize the Flask app
- Deploy the app to a free platform like Railway/Render (I can provide instructions)

Tell me which of these you want next.